In [ ]:
rs = 123

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rating_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/datasets/ratings.csv"
user_emb_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/datasets/user_embeddings.csv"
item_emb_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/datasets/course_embeddings.csv"

In [ ]:
rating_df = pd.read_csv(rating_url)
user_emb = pd.read_csv(user_emb_url)
item_emb = pd.read_csv(item_emb_url)

In [ ]:
merged_df = pd.merge(rating_df, user_emb, how='left', left_on='user', right_on='user').fillna(0)
merged_df = pd.merge(merged_df, item_emb, how='left', left_on='item', right_on='item').fillna(0)

In [ ]:
merged_df.head()

,user,item,rating,UFeature0,UFeature1,UFeature2,UFeature3,UFeature4,UFeature5,UFeature6,...,CFeature6,CFeature7,CFeature8,CFeature9,CFeature10,CFeature11,CFeature12,CFeature13,CFeature14,CFeature15
0,1889878,CC0101EN,3.0,0.080721,-0.129561,0.087998,0.030231,0.082691,-0.004176,-0.003480,...,-0.015081,-0.012229,0.015686,0.008401,-0.035495,0.009381,-0.032560,-0.007292,0.000966,-0.006218
1,1342067,CL0101EN,3.0,0.068047,-0.112781,0.045208,-0.007570,-0.038382,0.068037,0.114949,...,0.010899,-0.037610,-0.019397,-0.025682,-0.000620,0.038803,0.000196,-0.045343,0.012863,0.019429
2,1990814,ML0120ENv3,3.0,0.124623,0.012910,-0.072627,0.049935,0.020158,0.133306,-0.035366,...,-0.012695,0.036138,0.019965,0.018686,-0.010450,-0.050011,0.013845,-0.044454,-0.001480,-0.007559
3,380098,BD0211EN,3.0,-0.034870,0.000715,0.077406,0.070311,-0.043007,-0.035446,0.032846,...,-0.005700,-0.006068,-0.005792,-0.023036,0.015999,-0.023480,0.015469,0.022221,-0.023115,-0.001785
4,779563,DS0101EN,3.0,0.106414,-0.001887,-0.017211,-0.042277,-0.074953,-0.056732,0.074610,...,-0.010015,-0.001514,-0.017598,0.003590,0.016799,0.002732,0.005162,0.015031,-0.000877,-0.021283


In [ ]:
u_feautres = [f"UFeature{i}" for i in range(16)]
c_features = [f"CFeature{i}" for i in range(16)]

user_embeddings = merged_df[u_feautres]
course_embeddings = merged_df[c_features]
ratings = merged_df['rating']

interaction_dataset = user_embeddings + course_embeddings.values
interaction_dataset.columns = [f"Features{i}" for i in range(16)]
interaction_dataset['ratings'] = ratings
interaction_dataset.head()

,Features0,Features1,Features2,Features3,Features4,Features5,Features6,Features7,Features8,Features9,Features10,Features11,Features12,Features13,Features14,Features15,ratings
0,0.090378,-0.134799,0.083900,0.046534,0.077417,-0.004537,-0.018561,0.079236,-0.024561,0.027359,-0.188823,-0.080762,0.050271,-0.066013,0.058894,-0.007689,3.0
1,0.059437,-0.084740,0.067107,-0.009036,-0.031482,0.050057,0.125847,0.066517,-0.053798,-0.021671,0.064212,0.204660,-0.004188,0.007914,0.027170,0.076114,3.0
2,0.152061,-0.014739,-0.080112,-0.009516,0.024130,0.153802,-0.048061,-0.119888,0.059234,0.060882,0.004244,-0.166000,0.045002,0.057566,-0.022081,0.108929,3.0
3,-0.014707,-0.011257,0.073692,0.054763,-0.050547,-0.020599,0.027146,-0.067012,0.106593,-0.020921,0.106658,-0.092025,0.024436,0.086183,0.029232,0.016287,3.0
4,0.112812,-0.001395,-0.011572,-0.032638,-0.080440,-0.057321,0.064595,-0.020880,-0.048939,0.068486,-0.031359,-0.044577,-0.002381,0.025505,-0.033164,-0.105266,3.0


In [ ]:
X = interaction_dataset.iloc[:, :-1]
y_raw = interaction_dataset.iloc[:, -1]

In [ ]:
y_raw.value_counts()

ratings
3.0    222330
2.0     10976
Name: count, dtype: int64

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_raw.values.ravel())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X.shape, y.shape

((233306, 16), (233306,))

## TASK: Perform classification tasks on the interaction dataset
<br>
Now our input data `X` and output label `y` is ready, let's build classification models to map `X` to `y`
<br><br>
You may use `sklearn` to train and evaluate various regression models.
<br><br>
_TODO: Define classification models such as Logistic Regression, Tree models, SVM, Bagging, and Boosting models_


In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
predictions = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, predictions)

In [ ]:
accuracy

0.9533239038189534

## Try Different Models

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

In [24]:
def classification_model(X_train, y_train, X_test, y_test):

    #Logistic Regression
    log_reg = LogisticRegression()
    log_reg_params = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}
    log_reg_grid = GridSearchCV(log_reg, log_reg_params, cv=5)
    log_reg_grid.fit(X_train, y_train)
    log_reg_best = log_reg_grid.best_estimator_
    log_reg_accuracy = accuracy_score(y_test, log_reg_best.predict(X_test))



    # Decision Tree
    tree = DecisionTreeClassifier()
    tree_params = {'max_depth': [3, 5, 7], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 3, 5]}
    tree_grid = GridSearchCV(tree, tree_params, cv=5)
    tree_grid.fit(X_train, y_train)
    tree_best = tree_grid.best_estimator_
    tree_accuracy = accuracy_score(y_test, tree_best.predict(X_test))


     # Support Vector Machine
    svm = SVC()
    svm_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf'], 'gamma': ['scale', 'auto']}
    svm_grid = GridSearchCV(svm, svm_params, cv=5)
    svm_grid.fit(X_train, y_train)
    svm_best = svm_grid.best_estimator_
    svm_accuracy = accuracy_score(y_test, svm_best.predict(X_test))

    # Random Forest
    rf = RandomForestClassifier()
    rf_params = {'n_estimators': [100, 200, 300], 'max_depth': [None, 5, 10], 'min_samples_split': [2, 5, 10]}
    rf_grid = GridSearchCV(rf, rf_params, cv=5)
    rf_grid.fit(X_train, y_train)
    rf_best = rf_grid.best_estimator_
    rf_accuracy = accuracy_score(y_test, rf_best.predict(X_test))

    # Boosting
    ada_boost = AdaBoostClassifier()
    ada_params = {'n_estimators': [50, 100, 200],
                  'learning_rate': [0.01, 0.1, 1.0]}
    ada_grid = GridSearchCV(ada_boost, ada_params, cv=5)
    ada_grid.fit(X_train, y_train)
    ada_best = ada_grid.best_estimator_
    ada_accuracy = accuracy_score(y_test, ada_best.predict(X_test))

    results = {
        'Logistic Regression': log_reg_accuracy,
        'Decision Tree': tree_accuracy,
        'Support Vector Machine': svm_accuracy,
        'Random Forest': rf_accuracy,
        'Boosting': ada_accuracy
    }

    return results


In [ ]:
classification_model(X_train, y_train, X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve